In [50]:
import tqdm
import pandas as pd
from pathlib import Path
import xarray as xr
from spatial_temp_cgf.data import DEFAULT_ROOT, ClimateMalnutritionData

In [51]:
cm_data = ClimateMalnutritionData(DEFAULT_ROOT)

In [60]:
cm_data.load_fhs_shapes().set_index('loc_id').loc[482]

parent_id                                                   196
level                                                         4
loc_name                                           Eastern Cape
loc_nm_sh                                                E Cape
spr_reg_id                                                  166
region_id                                                   192
ihme_lc_id                                              ZAF_482
GAUL_CODE                                                 77310
area_sqkm                                         168905.174179
geometry      MULTIPOLYGON (((26.28513908000008 -33.83736037...
Name: 482, dtype: object

In [53]:
p = Path('/mnt/team/rapidresponse/pub/population/modeling/climate_malnutrition/stunting/results/2024_07_02.01/')

for scenario in ['ssp119', 'ssp245']:
    dfs = []
    for fp in tqdm.tqdm(list(p.glob(f"*_{scenario}.parquet"))):
        df = pd.read_parquet(fp)
        df['year_id'] = int(fp.name.split('_')[0])
        dfs.append(df)
    pd.concat(dfs).to_parquet(f'{scenario}.parquet')

idx_cols = ['location_id', 'age_group_id', 'sex_id', 'year_id']
ssp119 = pd.read_parquet('ssp119.parquet').set_index(idx_cols).value
ssp245 = pd.read_parquet('ssp245.parquet').set_index(idx_cols).value

combined = pd.concat([
    ssp245.rename('ssp245'),
    ssp119.rename('ssp119'),
], axis=1).sort_index()
combined = combined.reset_index().assign(location_id = lambda x: x.location_id.astype(int))
combined = combined.set_index(['location_id', 'age_group_id', 'sex_id', 'year_id']).sort_index()

100%|███████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 262.44it/s]


In [74]:
combined['delta_p'] = combined['ssp245'] - combined['ssp119']

In [76]:
combined['delta_p'].mean()

-3.0811461e-06

In [65]:
locs = combined.reset_index().location_id.unique()
len(locs)

1047

In [68]:
len(pop.reset_index().location_id.unique())

482

In [57]:
print(len(combined))
print(len(combined.index.difference(pop.index)))
print(list(combined.index.difference(pop.index))[:10])

339228
183060
[(482, 4, 1, 2020), (482, 4, 1, 2021), (482, 4, 1, 2022), (482, 4, 1, 2023), (482, 4, 1, 2024), (482, 4, 1, 2025), (482, 4, 1, 2026), (482, 4, 1, 2027), (482, 4, 1, 2028), (482, 4, 1, 2029)]


In [47]:
print(len(pop.index.difference(combined.index)))

3856


In [69]:
FORECASTED_POPULATIONS_FILEPATH = '/mnt/share/forecasting/data/7/future/population/20240529_500d_2100_lhc_ref_squeeze_hiv_shocks_covid_all_gbd_7_shifted/population.nc'
pop = xr.open_dataset(FORECASTED_POPULATIONS_FILEPATH).mean(dim='draw').sel(age_group_id=[4,5]).to_dataframe()#.reset_index("scenario", drop=True).reset_index().set_index(['location_id', 'age_group_id', 'sex_id', 'year_id'])

In [34]:
pop.reset_index().location_id.unique()

location_id       int64
age_group_id      int64
sex_id            int64
year_id           int64
population      float64
dtype: object

In [38]:
#list(combined.index.difference(pop.index))

In [39]:
#df = pd.concat([combined, pop.loc[combined.index]], axis=1)

In [ ]:
df['ssp245_count'] = df['ssp245'] * df['population']
df['ssp119_count'] = df['ssp119'] * df['population']
df['delta'] = df['ssp245_count'] - df['ssp119_count']

In [ ]:
df.groupby(['age_group_id', 'sex_id']).delta.sum()